In [5]:
from functools import partial
from pathlib import Path

import numpy as np
import torch.nn as nn
from rl_zoo3 import linear_schedule
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from wandb.integration.sb3 import WandbCallback

import wandb
from src.train.ea_ppo import make_env
from src.utils import load_config, save_config

In [2]:
config = {
    # Environment
    "action_mode": "delta",
    "max_quad_setting": 30.0,
    "max_quad_delta": 30.0,
    "max_steerer_delta": 6.1782e-3,
    "magnet_init_mode": np.array([10.0, -10.0, 0.0, 10.0, 0.0]),
    "incoming_mode": "random",
    "misalignment_mode": "random",
    "max_misalignment": 5e-4,
    "target_beam_mode": np.zeros(4),
    "threshold_hold": 1,
    "clip_magnets": True,
    # Wrappers
    "frame_stack": 1,  # 1 means no frame stacking
    "normalize_observation": True,
    "rescale_action": True,
    "target_threshold": None,  # 2e-5 m is estimated screen resolution
    "max_episode_steps": 50,
    # RL algorithm
    "batch_size": 64,
    "learning_rate": 0.0003,
    "lr_schedule": "constant",  # Can be "constant" or "linear"
    "gamma": 0.99,
    "n_envs": 40,
    "n_steps": 64,
    "ent_coef": 0.0,
    "n_epochs": 10,
    "gae_lambda": 0.95,
    "clip_range": 0.2,
    "clip_range_vf": None,  # None,
    "vf_coef": 0.5,
    "max_grad_norm": 0.5,
    "use_sde": False,
    "sde_sample_freq": -1,
    "target_kl": None,
    "total_timesteps": 500_000,
    # Policy
    "net_arch": "small",  # Can be "small" or "medium"
    "activation_fn": "Tanh",  # Tanh, ReLU, GELU
    "ortho_init": True,  # True, False
    "log_std_init": 0.0,
    # SB3 config
    "sb3_device": "auto",
    "vec_env": "subproc",
}

In [3]:
# Setup wandb
wandb.init(
    entity="msk-ipc",
    project="rl4aa-tutorial-2025-dev",
    sync_tensorboard=True,
    monitor_gym=True,
    config=config,
    dir=".wandb",
)
config = dict(wandb.config)
config["run_name"] = wandb.run.name

# Setup vecorised environments for training and evaluation
assert config["vec_env"] in ["dummy", "subproc"]
vec_env = (
    DummyVecEnv([partial(make_env, config) for _ in range(config["n_envs"])])
    if config["vec_env"] == "dummy"
    else SubprocVecEnv([partial(make_env, config) for _ in range(config["n_envs"])])
)
eval_vec_env = DummyVecEnv(
    [partial(make_env, config, plot_episode=True, log_task_statistics=True)]
)


# Setup learning rate schedule if needed
if config["lr_schedule"] == "linear":
    config["learning_rate"] = linear_schedule(config["learning_rate"])

# Setup RL training algorithm
model = PPO(
    "MlpPolicy",
    vec_env,
    learning_rate=config["learning_rate"],
    n_steps=config["n_steps"],
    batch_size=config["batch_size"],
    n_epochs=config["n_epochs"],
    gamma=config["gamma"],
    gae_lambda=config["gae_lambda"],
    clip_range=config["clip_range"],
    clip_range_vf=config["clip_range_vf"],
    ent_coef=config["ent_coef"],
    vf_coef=config["vf_coef"],
    max_grad_norm=config["max_grad_norm"],
    use_sde=config["use_sde"],
    sde_sample_freq=config["sde_sample_freq"],
    target_kl=config["target_kl"],
    policy_kwargs={
        "activation_fn": getattr(nn, config["activation_fn"]),
        "net_arch": {  # From rl_zoo3
            "small": {"pi": [64, 64], "vf": [64, 64]},
            "medium": {"pi": [256, 256], "vf": [256, 256]},
        }[config["net_arch"]],
        "ortho_init": config["ortho_init"],
        "log_std_init": config["log_std_init"],
    },
    device=config["sb3_device"],
    tensorboard_log=f"log/{config['run_name']}",
    verbose=1,
)

# Setup callbacks for evaluation and logging
eval_callback = EvalCallback(eval_vec_env, eval_freq=1_000, n_eval_episodes=5)
wandb_callback = WandbCallback()

# Train the model
model.learn(
    total_timesteps=config["total_timesteps"],
    callback=[eval_callback, wandb_callback],
)

# Save the model and associated configuration
model.save(f"models/ea/ppo/{wandb.run.name}/model")
save_config(config, f"models/ea/ppo/{wandb.run.name}/config")

# Cleanup environments
vec_env.close()
eval_vec_env.close()

Using cpu device
Logging to log/dummy-lbjquxgd/PPO_1


/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/site-packages/stable_baselines3/common/callbacks.py:418: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x16a5e3ef0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x3020a2960>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -0.744   |
| time/              |          |
|    fps             | 1103     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2560     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 50         |
|    ep_rew_mean          | -0.736     |
| time/                   |            |
|    fps                  | 1401       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 5120       |
| train/                  |            |
|    approx_kl            | 0.01133293 |
|    clip_fraction        | 0.124      |
|    clip_range           | 0.2        |
|    entropy_loss         | -7.1       |
|    explained_variance   | -2.67      |
|    learning_rate        | 0.0003     |
|   

/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'models/ea/ppo/dummy-lbjquxgd' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
/Users/jankaiser/Documents/DESY/rl4aa25-tutorial/src/wrappers/plot_episode.py:128: UserWarning: WARN: Unable to save episode plot for self.episode_id = 60 because the episode was too short.
  logger.warn(
/Users/jankaiser/Documents/DESY/rl4aa25-tutorial/src/wrappers/log_task_statistics.py:72: UserWarning: WARN: Unable to save episode plot for self.episode_id = 60 because the episode was too short.
  logger.warn(


In [6]:
model_name = "feasible-plasma-476"
model_path = Path("models") / "ea" / "ppo" / model_name
model = PPO.load(model_path / "model")
config = load_config(model_path / "config")

/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
